<a href="https://colab.research.google.com/github/sayan0506/Face-Recognition-pipeline-using-SOTA-ArcFace/blob/main/Face_recognition_using_arcface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Face recognition Pipeline design using arcface**

## **Import Dependencies**

In [8]:
from pathlib import Path
import numpy as np
import cv2

from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile

# ensures loading truncated images 
ImageFile.LOAD_TRUNCATED_IMAGES = True